# Using Custom Anomaly Signatures
This notebook shows how to use the `AnomalyNearestNeighbor` class to quantify custom
heat flow anomaly signatures (say derived from finite element analysis).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from reheatfunq.regional import default_prior, HeatFlowPredictive
from reheatfunq.anomaly import HeatFlowAnomalyPosterior, AnomalyNearestNeighbor

Generate a custom anomaly signature. Here we would like to generate an Gaussian anomaly
centered at $(x,y)=(0,0)$, which leads to a heat flow offset of $68.3\,\mathrm{mW\,m}^{-2}$
in its center when powered by $10\,\mathrm{MW}$. Noting that the $c_i$ should be given in
SI basis units ($\mathrm{m}^{-2}$), we find the following expression:

In [ ]:
def anomaly_ci(x,y):
    return 68.3e-3 / 10e6 * np.exp(-(x**2 + y**2))

"Load" the heat flow data:

In [ ]:
N = 20
rng = np.random.default_rng(123329773)
xy = 3 * rng.random((N,2)) - 1.5
q0 = 0.39 * rng.gamma(175.2, size=N)
c_i = anomaly_ci(*xy.T)
q = q0 + 1e3 * 10e6 * c_i

To customize the analysis, `xy` and `q` needs to be replaced with heat flow data and `c_i` would have to be computed as desired.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.step(np.sort(q), np.arange(q.size)/(q.size-1))
ax.set_xlabel('$q$ ($\\mathrm{mW\,m}^{-2}$)')
ax.set_ylabel('CDF');

Plot the configuration:

In [ ]:
xp = np.linspace(-1.5, 1.5, 51)
yp = np.linspace(-1.5, 1.5)
xg, yg = np.meshgrid(xp, yp)
ano_g = anomaly_ci(xg,yg)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')
cset = ax.contour(xp, yp, ano_g, cmap='magma')
h = ax.scatter(*xy.T, c=q)
fig.draw_without_rendering()
ax.clabel(cset, zorder=0, fmt="%1.0e")
ax.set_xlabel('x')
ax.set_ylabel('y')
fig.colorbar(h, label='$q$');

### REHEATFUNQ Analysis
Now perform a default REHEATFUNQ analysis. For simplicity, we use $d_\mathrm{min}=0$ here.

In [ ]:
anomaly = AnomalyNearestNeighbor(np.stack((*xy.T, c_i), axis=1))

In [ ]:
gcp = default_prior()

In [ ]:
hfap = HeatFlowAnomalyPosterior(q, *xy.T, anomaly, gcp, dmin=0.0)

In [ ]:
fig = plt.figure()
P_H = np.linspace(6e6, 1.5e7, 200)
y = hfap.pdf(P_H)
ax = fig.add_subplot(111)
ax.plot(1e-6*P_H, y, label='Posterior PDF')
ax.axvline(10, color='k', linestyle='--', linewidth=1.0, label='True anomaly')
ax.set_ylim(0, ax.get_ylim()[1])
ax.set_xlabel('Frictional power $P_H$ (MW)')
ax.set_ylabel('Posterior density ($\mathrm{W}^{-1}$)');
ax.legend();

### License
```
REHEATFUNQ custom heat flow anomaly analysis quickstart notebook.

This file is part of the REHEATFUNQ model.

Author: Malte J. Ziebarth (ziebarth@gfz-potsdam.de)

Copyright © 2022 Malte J. Ziebarth

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
```